## Hydropedology Model Project 1

This is the starter for the hydropedology modelling project in Winter Term 2022 at TUBAF. We aim at modelling the hydrology for the inflow of the following drinking water reservoirs in Western Ore Mountains:
 * [Muldenberg](https://www.ltv.sachsen.de/tmz/pegel/506.html)
 * [Werda](https://www.ltv.sachsen.de/tmz/pegel/503.html)
 * [Sosa](https://www.ltv.sachsen.de/tmz/pegel/402.html)
 * [Carlsfeld](https://www.ltv.sachsen.de/tmz/pegel/401.html)

We aim at about hourly modelling. If this turns out to be too far fetched, we will reduce to daily versions. Moreover, we will first perform rainfall-runoff-modelling. We will then extend these models to EC and DOC signals. We will use different model candidates starting with [SuperFlex](https://superflexpy.readthedocs.io/en/latest/index.html) and going further to [NeuralHydrology](https://neuralhydrology.readthedocs.io/en/latest/index.html). Probably, further models will be used along our journey.

To get there, we will have to:
 1. Resource all data needed for modelling
 2. Get familiar with the data and the models
 3. Prepare data and models for calibration and evaluation (using [SPOTPY](https://spotpy.readthedocs.io/en/latest/))
 
Have fun. 

(cc) conrad.jackisch@tbt.tu-freiberg.de, Nov 01, 2022

In [1]:
%pylab inline
import pandas as pd
import seaborn as sns
import plotly.express as px

import plotly.io as pio
pio.renderers.default='iframe' #for windows use 'notebook' instead
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from plotly.figure_factory import create_distplot

sns.set_style('whitegrid', {'grid.linestyle': u'--'})
matplotlib.rc('pdf', fonttype=42)

Populating the interactive namespace from numpy and matplotlib


In [2]:
import dwd
import pyeto as pt
import meteostat as mt

## DWD Stationen mit unterschiedlicher Datenverfügbarkeit

In [ ]:
# DWD Stations
dwd_stats = dwd.dwd_stations()
dwd_stats['StatID'] = dwd_stats.index
dwd_stats['Active'] = np.log(((pd.Timestamp.now()-dwd_stats['bis_datum']).values).astype(float)/86400000000000.)

In [4]:
px.set_mapbox_access_token('pk.eyJ1IjoiY29qYWNrIiwiYSI6IkRTNjV1T2MifQ.EWzL4Qk-VvQoaeJBfE6VSA')

fig = px.scatter_mapbox(dwd_stats, lat='geoBreite', lon='geoLaenge',  color='Active', hover_name='Stationsname', hover_data=['StatID','von_datum', 'bis_datum'],
                  color_continuous_scale=px.colors.sequential.YlOrRd_r, zoom=5)
fig.show()

You can download the DWD data with my toolbox by simply defining the station through its name. You can select to download only recent data `histo = False` or the complete set provided on the DWD server `histo = False`. You can choose daily `rss='1D'` and hourly data `rss='1H'`.

In [4]:
# load station weather data

#weather = dwd.resample_DWD('Carlsfeld',histo=True,rss='1H')
#weather.to_csv('carlsfeld_dwd.csv')
weather = pd.read_csv('Carlsfeld_dwd.csv', index_col=0)
weather.index = pd.to_datetime(weather.index)

In [6]:
weather

,T,Tmin,Tmax,Prec,Rad,RHmin,RHmax,u2,aP
1990-05-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
1990-05-01 01:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN
1990-05-01 02:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
1990-05-01 03:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN
1990-05-01 04:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN
...,...,...,...,...,...,...,...,...,...
2022-10-30 19:00:00,10.7,10.7,10.7,0.0,NaN,70.0,70.0,1.9,-999.0
2022-10-30 20:00:00,10.5,10.5,10.5,0.0,NaN,69.0,69.0,1.5,-999.0
2022-10-30 21:00:00,10.1,10.1,10.1,0.0,NaN,73.0,73.0,1.9,-999.0
2022-10-30 22:00:00,11.0,11.0,11.0,0.0,NaN,66.0,66.0,2.3,-999.0


In [7]:
carlsfeld_ap = pd.read_csv('produkt_p0_stunde_19900611_20211231_00840.txt',na_values=('-999','eor'),sep=';')
carlsfeld_ap[carlsfeld_ap==-999.0] = np.nan
carlsfeld_ap.index = pd.to_datetime(carlsfeld_ap.MESS_DATUM, format='%Y%m%d%H')
carlsfeld_ap

,STATIONS_ID,MESS_DATUM,QN_8,P,P0,eor
MESS_DATUM,,,,,,
1990-06-11 14:00:00,840,1990061114,1,NaN,NaN,NaN
1990-06-11 15:00:00,840,1990061115,1,NaN,NaN,NaN
1990-06-11 16:00:00,840,1990061116,1,NaN,NaN,NaN
1990-06-11 17:00:00,840,1990061117,1,NaN,NaN,NaN
1990-06-11 18:00:00,840,1990061118,1,NaN,NaN,NaN
...,...,...,...,...,...,...
2021-12-31 19:00:00,840,2021123119,3,NaN,915.9,NaN
2021-12-31 20:00:00,840,2021123120,3,NaN,916.6,NaN
2021-12-31 21:00:00,840,2021123121,3,NaN,916.7,NaN


In [18]:
dummyk = pd.DataFrame(carlsfeld_ap['  P0'])
dummyk.columns = ['P0']

weather1 = pd.concat([dummyk,weather], axis=1)
weather1.columns# = ['P0', 'Prec', 'RHmax', 'RHmin', 'Rad', 'T', 'Tmax', 'Tmin', 'aP', 'u2']
#px.line(weather1[['P0','aP']], template='none')

Index(['P0', 'T', 'Tmin', 'Tmax', 'Prec', 'Rad', 'RHmin', 'RHmax', 'u2', 'aP'], dtype='object')

In [39]:
weather1D = weather.resample('1D').agg({'T': 'mean', 'Tmin': 'min', 'Tmax': 'max', 'Prec': 'sum', 'Rad': 'sum', 'RHmin': 'min', 'RHmax': 'max','u2': 'mean', 'aP':'mean'})


In [1]:
import pyet
pyet.pm_fao56(weather1D['T'], rn=weather1D['Rad'], wind=weather1D['u2'], tmax=weather1D['Tmax'], tmin=weather1D['Tmin'], 
                         lat=51., elevation=900,
                         rhmin=weather1D['RHmin'], rhmax=weather1D['RHmax'])

NameError: name 'weather1D' is not defined

In [44]:
ET0SJ, ET0PM, ET0PT = dwd.ET_SzilagyiJozsa(weather1D,Elev = 900., lat = 50.42)
dummyk = pd.concat([ET0SJ, ET0PM, ET0PT],axis=1)
dummyk.columns = ['ET0SJ', 'ET0PM', 'ET0PT']
weather1D = pd.concat([weather1D,dummyk],axis=0)


In [46]:
px.line(weather1D[['Prec','ET0PM']], template='none')

In [5]:
#EToPM = pt.fao56_penman_monteith(weather.Rs.values,weather['T'].values+273.15,
#                                 weather.u2.values,pt.svp_from_t(weather['T'].values),
#                                 pt.avp_from_rhmin_rhmax(pt.svp_from_t(weather.Tmin.values), pt.svp_from_t(weather.Tmax.values), weather.RHmin.values, weather.RHmax.values),
#                                 pt.delta_svp(weather['T'].values),
#                                 pt.psy_const(weather.aP.values*0.1))
#

,T,Tmin,Tmax,Prec,Rad,RHmin,RHmax,u2,aP
1990-05-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
1990-05-01 01:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN
1990-05-01 02:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
1990-05-01 03:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN
1990-05-01 04:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN
...,...,...,...,...,...,...,...,...,...
2022-10-30 19:00:00,10.7,10.7,10.7,0.0,NaN,70.0,70.0,1.9,-999.0
2022-10-30 20:00:00,10.5,10.5,10.5,0.0,NaN,69.0,69.0,1.5,-999.0
2022-10-30 21:00:00,10.1,10.1,10.1,0.0,NaN,73.0,73.0,1.9,-999.0
2022-10-30 22:00:00,11.0,11.0,11.0,0.0,NaN,66.0,66.0,2.3,-999.0


In [ ]:
#https://www.geodaten.sachsen.de/downloadbereich-digitale-hoehenmodelle-4851.html

In [106]:
#carlsfeld_ap.columns

Index(['STATIONS_ID', 'MESS_DATUM', 'QN_8', '   P', '  P0', 'eor'], dtype='object')

In [111]:
#px.line(carlsfeld_ap['  P0'], template='none')

In [96]:
weather[weather == -999.0] = np.nan
px.line(weather.aP, template='none')

In [112]:
#weather.to_csv('Carlsfeld_dwd.csv')

weather = pd.read_csv('Carlsfeld_dwd.csv', index_col=0)
weather.index = pd.to_datetime(weather.index)
weather[weather == -999.0] = np.nan
weather

,T,Tmin,Tmax,Prec,Rad,RHmin,RHmax,u2,aP
1990-05-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
1990-05-01 01:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN
1990-05-01 02:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
1990-05-01 03:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN
1990-05-01 04:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN
...,...,...,...,...,...,...,...,...,...
2022-10-30 19:00:00,10.7,10.7,10.7,0.0,NaN,70.0,70.0,1.9,NaN
2022-10-30 20:00:00,10.5,10.5,10.5,0.0,NaN,69.0,69.0,1.5,NaN
2022-10-30 21:00:00,10.1,10.1,10.1,0.0,NaN,73.0,73.0,1.9,NaN
2022-10-30 22:00:00,11.0,11.0,11.0,0.0,NaN,66.0,66.0,2.3,NaN


In [114]:
weather = pd.concat([weather, carlsfeld_ap['  P0']],axis=1)

In [115]:
px.line(weather,x=weather.index, y=['T','Rad','  P0'])

Check the manual for [meteostat](https://dev.meteostat.net/python/hourly.html#api) also for the units contained.

In [92]:
#Hourly Data Carlsfeld
# Time period
start = pd.to_datetime('1991-01-01 00:00')
end = pd.to_datetime('2022-01-10 00:00')

# Get closest weather station
stations = mt.Stations()
stations = stations.nearby(50.95, 12.71) #coord Aue 50.479250, 12.651017) #coords Sosa
k = 1
while all(mt.Hourly(stations.fetch(k), start, end).fetch().temp.isna()):
    k += 1
station = stations.fetch(k)

# Get hourly data
idx = station.index[-1]
data = mt.Hourly(stations.fetch(k), start, end).fetch()
data.columns

Index(['temp', 'dwpt', 'rhum', 'prcp', 'snow', 'wdir', 'wspd', 'wpgt', 'pres',
       'tsun', 'coco'],
      dtype='object')

In [93]:
data

,temp,dwpt,rhum,prcp,snow,wdir,wspd,wpgt,pres,tsun,coco
time,,,,,,,,,,,
2008-06-01 00:00:00,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2008-06-01 01:00:00,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2008-06-01 02:00:00,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2008-06-01 03:00:00,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2008-06-01 04:00:00,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2022-01-09 20:00:00,0.8,-0.5,91.0,0.0,NaN,241.0,14.4,NaN,1006.8,NaN,NaN
2022-01-09 21:00:00,0.9,-0.6,90.0,0.0,NaN,247.0,13.7,NaN,1007.6,NaN,NaN
2022-01-09 22:00:00,1.0,-0.6,89.0,0.0,NaN,250.0,13.0,NaN,1008.3,NaN,NaN


In [89]:
px.line(data.pres, template = 'none')

In [36]:
#px.scatter(data.temp.resample('1Y').mean().iloc[:-1], trendline='ols', template='none')
#px.line(data.tsun.resample('1m').sum(), template='none')
px.line(radsun(data), template='none')